In [ ]:
#default_exp random

In [ ]:
#export 
from kesscore.imports import *
from kesscore.functional import *

In [ ]:
#export
def _rangechoice(len,size=None,nprs=None,replace=None,cum_weights=None,p=None):
    k,pyrs = ifnone(size,1),random.Random(nprs.random())
    if replace:
        assert p is None,'Should use cum_weights'
        if cum_weights is None: idx = nprs.randint(len,size=k)
        else:                   idx = pyrs.choices(range(len),k=k,cum_weights=cum_weights) 
                                     # ^ here can also provide weight=p but performencewise not.
    else:
        assert cum_weights is None,'Should use p'
        if p is None: idx = pyrs.sample(range(len),k)
        else:         idx = nprs.choice(np.arange(len),size=k,replace=False,p=p)
    return idx[0] if size == None else idx
        
@delegates(_rangechoice, but=['len'])
def _customrangechoice(rng,**kwargs):
    idx = _rangechoice(len(rng),**kwargs)
    if not isinstance(idx, Iterable): return rng[idx]
    else: return np.asarray(idx) * rng.step + rng.start

@typedispatch
def _choicedsp(a,**kwargs):                  raise ValueError(f'unknown type {type(a)}')
@typedispatch
def _choicedsp(a:typing.Dict,**kwargs):      return _choicedsp(list(a), **kwargs)
@typedispatch
def _choicedsp(a:numbers.Integral,**kwargs): return _choicedsp(range(a),**kwargs)
@typedispatch
def _choicedsp(a:slice,**kwargs):            return _choicedsp(range(a.start,a.stop,a.step),**kwargs)
@typedispatch
def _choicedsp(a:range,**kwargs):            return _customrangechoice(a,**kwargs)
@typedispatch
def _choicedsp(a:(np.ndarray,torch.Tensor,L),**kwargs): idx=_rangechoice(len(a),**kwargs); return a[idx]
@typedispatch
def _choicedsp(a:(list, tuple),**kwargs):
    idx = _rangechoice(len(a),**kwargs); 
    return a[idx] if kwargs['size'] is None else lmap(lambda i:a[i], idx)

@delegates(_rangechoice)
def _choice(a,nprs,size=None,**kwargs): return _choicedsp(a,nprs=nprs,size=size,**kwargs)

In [ ]:
def _get_rs(seed=10): return np.random.RandomState(seed)

def testc(sz,rp,cw=None,p=None): return _choice([1,2,3,5,6,7],_get_rs(),sz,replace=rp,cum_weights=cw,p=p)
test_eq_type(testc(None,False),2)
test_eq_type(testc(None,True),6)
test_eq_type(testc(1,True),[6])
test_eq_type(testc(10,True), [6, 1, 2, 5, 6, 2, 7, 1, 7, 2])
test_eq_type(testc(6,False), [2, 6, 3, 7, 5, 1])
test_fail(lambda:testc(7,False),contains='Sample larger than population or is negative')

test_eq_type(_choice({1:2}, _get_rs()), 1)
test_eq_type(_choice(100, _get_rs(), 3), np.array([21, 76, 33]))
test_fail(lambda:_choice('ido',_get_rs()), contains='unknown type <class \'str\'>')

a = np.random.RandomState(10).randn(10,3)
def testa(sz,rp,cw=None,p=None): return _choice(a,_get_rs(),sz,replace=rp,cum_weights=cw,p=p)
test_eq_type(testa(None,True), a[4])
test_eq_type(testa(1,True), a[[4]])
testa(3,True,cw=[0.1] + [0.0]*9)
p = (np.arange(10)+1)**3 
p = p.astype(np.float64)  / p.sum()
pcum = p.cumsum()

test_eq_type(testa(3, True), a[[4,0,1]])
test_eq_type(testa(3, True, cw=pcum), a[[-1,-1,-1]])
test_eq_type(testa(3, False, p=p), a[[3,8,9]])

In [ ]:
#export
@docs
class RandomState:
    def __init__(self, seed=None): self.rs=np.random.RandomState(seed=seed)
    def perm(self,a): return self.rs.permutation(range(a) if isinstance(a,numbers.Integral) else a)
    def choice(self,a,size=None,cum_weights=None): return _choice(a,self.rs,size=size,replace=True, cum_weights=cum_weights)
    def sample(self,a,size=None,p=None):           return _choice(a,self.rs,size=size,replace=False,p=p)
    def randn(self,size=None):return self.rs.randn(*size)
    def rand(self,size=None): return self.rs.random(size=size)
    def bool(self,size=None,p=0.5): return self.rand(size) < p
    def int(self,low,high=None,size=None,dtype=int): return self.rs.randint(low=low,high=high,size=size,dtype=dtype)
    _docs = dict(
        cls_doc = 'RandomState numpy\pythonic. if seed not given, randomly select one.',
        perm    = 'Return a random permutation',
        choice  = 'Get sample with replacements',
        sample  = 'Get sample w.o  replacements',
        randn   = 'Return floats in [0, 1)',
        rand    = 'Return floats from normal distribution N(0,1)',
        bool    = 'Return True with probability p',
        int     = 'Return int(ndarray) with in [low, high) ([0, low))',
    )

In [ ]:
test_eq_type(RandomState(10).perm(10), np.array([8, 2, 5, 6, 3, 1, 0, 7, 4, 9]))
test_eq_type(RandomState(9).perm([[1,2],[3,4]]),np.array([[3, 4],[1, 2]]))

In [ ]:
test_eq(RandomState(10).bool(size=10), [False,True,False,False,True,True,True,False,True,True])
test_eq_type(RandomState(10).bool(), False)
test_eq_type(RandomState(10).bool(p=0.9), True)
test_eq_type(RandomState(10).bool(p=0.1), False)
test_eq(RandomState(10).choice([1,2,3]), 1)
test_eq(RandomState(10).choice({1:2, 3:4}, 3), [1, 3, 1])
test_eq(RandomState(10).sample({1:2, 3:4}, 2), [1,3])

test_fail(lambda: RandomState(10).sample([1,2], 3), contains='Sample larger than population or is negative')
test_eq(np.round(RandomState(10).rand(3), 3), [0.771, 0.021, 0.634])
test_eq(RandomState(10).int(9,size=4), [4, 0, 1, 0])

test_eq(RandomState(10).int(9,size=1000).max(), 8)
test_eq(RandomState(10).int(9,size=1000).min(), 0)

test_eq(RandomState(10).int(1, 9,size=1000).max(), 8)
test_eq(RandomState(10).int(1, 9,size=1000).min(), 1)

In [ ]:
from nbdev.sync import notebook2script

In [ ]:
notebook2script()

Converted 00_functional.ipynb.
Converted 01_images.ipynb.
Converted 02_fastdl.ipynb.
Converted 03_datadownload.ipynb.
Converted 04_random.ipynb.
Converted index.ipynb.
